In [4]:
import pandas as pd
import fastf1 as ff1
from datetime import timedelta

In [5]:
# Configurazione della sessione
anno = 2023
gara = 'Monaco'
sessione = 'R'  # Race

In [6]:
# Carica la sessione
session = ff1.get_session(anno, gara, sessione)
session.load()

req         WARNING 	DEFAULT CACHE ENABLED! (818.97 MB) C:\Users\calin\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '31', '44', '63', '16', '10', '55', '4', '81', '77', '21', '24', '23', 

In [ ]:
# Funzione per convertire timedelta in formato MM:SS
def format_minute_second(td):
    total_seconds = int(td.total_seconds())
    minutes, seconds = divmod(total_seconds, 60)
    return f"{minutes:02}:{seconds:02}"

# Ottieni i dati meteo
weather_data = session.weather_data

# Ottieni i dati del pilota (esempio: Verstappen)
driver = 'HAM'
driver_laps = session.laps.pick_drivers([driver])
driver_telemetry = driver_laps.get_car_data().add_distance()

# Ottieni informazioni sul circuito
circuito = session.event.Location  # Nome del circuito

# Nota: FastF1 non fornisce direttamente la lunghezza del circuito, quindi usiamo una mappatura manuale
lunghezza_circuito = {
    'Melbourne': 5278, #lunghezza circiuito in metri
    'Shanghai': 5451,
    'Suzuka': 5807,
    'Bahrain': 5412,
    'Jeddah': 6174,
    'Miami': 5412,
    'Imola': 4909,
    'Monaco': 3337,
    'Spagna': 4657,
    'Montreal': 4361,
    'Austria': 4318,
    'Silverstone': 5891,
    'Spa': 7004,
    'Hungaroring': 4381,
    'Zandvoort': 4259,
    'Monza': 5793,
    'Baku': 6003,
    'Singapore': 4940,
    'Austin': 5513,
    'Mexico City': 4304,
    'Interlagos': 4309,
    'Las Vegas': 6201,
    'Qatar': 5419,
    'Abu Dhabi': 5281,
    'Le Castellet': 5842,
    'Sochi': 5848,
    'Portimao': 4653,
    'Istanbul': 5338,
    'Nurburgring': 5148,
    'Hockenheim': 4574,
    #circuiti vecchi usati prima del 2018
    'Adelaide': 3780,
    'Phoenix': 3770,
    'Estoril': 4415,
    'Magny-Cours': 4411,
    'Indianapolis': 4180,
    'Sepang': 5543,
    'Valencia': 4005,
    'Yeongam': 5603,
    'Jerez': 4423,
    'Kyalami': 4300,
    'Estoril': 4415,
    'Montjuic': 3200,
    'Zolder': 4216,
    'Brands Hatch': 3962,
    'Cyril': 4000,
    'Donington Park': 4023,
    'Fuji': 4571
}.get(circuito, 0) 

# Trovo i giri totali del pilota
giri_totali = session.total_laps

# Trova la scuderia del pilota
scuderia = driver_laps['Team'].iloc[0]

# Ottieni i dati di telemetria per tutti i piloti
all_drivers_telemetry = {}
for drv in session.drivers:
    drv_laps = session.laps.pick_drivers([drv])
    if not drv_laps.empty:
        all_drivers_telemetry[drv] = drv_laps.get_car_data().add_distance()


# Crea un intervallo di timestamp ogni 0.5 secondi
start_time = timedelta(seconds=0)
end_time = weather_data['Time'].max()  # Durata massima della sessione
timestamp_intervals = pd.timedelta_range(start=start_time, end=end_time, freq='500L')  # 500 millisecondi

# Crea un DataFrame vuoto con le nuove colonne
df = pd.DataFrame(columns=[
    'Timestamp',
    'Driver', 'Scuderia', 'Track_Temperature', 'Speed',
    'Anno', 'Circuito','Giri_Totali', 'Lunghezza_Circuito', 'Posizione','Giro_in_Corso','Distanza_dal_precedente', 'Distanza_dal_successivo','Metri_Percorsi', 'Percentuale_Completamento_Giro',
    'Tempo_Trascorso_Dal_Giro','Temperatura_Esterna', 'Pioggia', 'Tipo_Gomma' ])

# Itera sui timestamp e raccogli i dati
for timestamp in timestamp_intervals:

    # Trova la temperatura della pista più vicina nel tempo
    nearest_weather_idx = (weather_data['Time'] - timestamp).abs().idxmin()
    track_temperature = weather_data.loc[nearest_weather_idx, 'TrackTemp']
    air_temperature = weather_data.loc[nearest_weather_idx, 'AirTemp']  # Temperatura esterna

    # Determina se piove
    rainfall = weather_data.loc[nearest_weather_idx, 'Rainfall']
    pioggia = 1 if rainfall else 0  # 1 = piove, 0 = non piove

    # Trova la velocità del pilota più vicina nel tempo
    nearest_telemetry_idx = (driver_telemetry['Time'] - timestamp).abs().idxmin()
    speed = driver_telemetry.loc[nearest_telemetry_idx, 'Speed']

     # Trova la posizione del pilota più vicina nel tempo
    nearest_position_idx = (driver_laps['Time'] - timestamp).abs().idxmin()
    posizione = driver_laps.loc[nearest_position_idx, 'Position']

     # Trova il giro in corso più vicino nel tempo
    nearest_lap_idx = (driver_laps['LapStartTime'] - timestamp).abs().idxmin()
    giro_in_corso = driver_laps.loc[nearest_lap_idx, 'LapNumber']

    # Trova il tipo di gomma utilizzato durante il giro in corso
    giro_in_corso = driver_laps.loc[nearest_lap_idx, 'LapNumber']
    tipo_gomma = driver_laps[driver_laps['LapNumber'] == giro_in_corso]['Compound'].iloc[0]

 # Trova la distanza dal pilota precedente
    if posizione == 1:
        distanza_dal_precedente = 0  # Se è primo, la distanza è 0
    else:
        # Trova il pilota precedente in classifica
        pilota_precedente = None
        for drv, telemetry in all_drivers_telemetry.items():
            drv_position = session.laps.pick_drivers([drv])['Position'].iloc[0]
            if drv_position == (posizione - 1):
                pilota_precedente = telemetry
                break

        if pilota_precedente is not None:
            # Trova la distanza tra il pilota e quello precedente
            nearest_prec_idx = (pilota_precedente['Time'] - timestamp).abs().idxmin()
            distanza_dal_precedente = driver_telemetry.loc[nearest_telemetry_idx, 'Distance'] - \
                                      pilota_precedente.loc[nearest_prec_idx, 'Distance']
        else:
            distanza_dal_precedente = 0  # Se non ci sono dati, usa 0

    # Trova la distanza dal pilota successivo
    if posizione == len(session.drivers):
        distanza_dal_successivo = 0  # Se è ultimo, la distanza è 0
    else:
        # Trova il pilota successivo in classifica
        pilota_successivo = None
        for drv, telemetry in all_drivers_telemetry.items():
            drv_position = session.laps.pick_drivers([drv])['Position'].iloc[0]
            if drv_position == (posizione + 1):
                pilota_successivo = telemetry
                break

        if pilota_successivo is not None:
            # Trova la distanza tra il pilota e quello successivo
            nearest_succ_idx = (pilota_successivo['Time'] - timestamp).abs().idxmin()
            distanza_dal_successivo = pilota_successivo.loc[nearest_succ_idx, 'Distance'] - \
                                      driver_telemetry.loc[nearest_telemetry_idx, 'Distance']
        else:
            distanza_dal_successivo = 0  # Se non ci sono dati, usa 0
    # Calcola i metri percorsi e la percentuale di completamento del giro
    metri_percorsi = driver_telemetry.loc[nearest_telemetry_idx, 'Distance']
    percentuale_completamento = (metri_percorsi % lunghezza_circuito) / lunghezza_circuito * 100
    
    # Aggiungi i dati al DataFrame
    df.loc[len(df)] = [
        timestamp,  # Timestamp
        driver,     # Nome del pilota
        scuderia,   # Scuderia del pilota
        track_temperature,  # Temperatura della pista
        speed,  # Velocità del pilota
        anno,  # Anno
        circuito,  # Nome del circuito
        giri_totali, # Giri totdel pilota
        lunghezza_circuito,  # Lunghezza del circuito in metri
        posizione, # Posizione del pilota
        giro_in_corso, # Giro in corso del pilota
        distanza_dal_precedente,  # Distanza dal pilota precedente
        distanza_dal_successivo,  # Distanza dal pilota successivo
        metri_percorsi,  # Metri percorsi
        percentuale_completamento,  # Percentuale di completamento del giro
        tempo_trascorso_dal_giro,  # Tempo nel giro
        air_temperature,  # Temperatura esterna
        pioggia,  # Pioggia (0 = non piove, 1 = piove)
        tipo_gomma  # Tipo di gomma utilizzato
    ]

# Visualizza il DataFrame risultante
print(df.head(10))

C:\Users\calin\AppData\Local\Temp\ipykernel_12732\4227143983.py:81: FutureWarning: 'L' is deprecated and will be removed in a future version, please use 'ms' instead.
  timestamp_intervals = pd.timedelta_range(start=start_time, end=end_time, freq='500L')  # 500 millisecondi


               Timestamp Driver  Scuderia  Track_Temperature  Speed  Anno  \
0        0 days 00:00:00    HAM  Mercedes               47.4      0  2023   
1 0 days 00:00:00.500000    HAM  Mercedes               47.4      0  2023   
2        0 days 00:00:01    HAM  Mercedes               47.4     12  2023   
3 0 days 00:00:01.500000    HAM  Mercedes               47.4     30  2023   
4        0 days 00:00:02    HAM  Mercedes               47.4     50  2023   
5 0 days 00:00:02.500000    HAM  Mercedes               47.4     67  2023   
6        0 days 00:00:03    HAM  Mercedes               47.4     78  2023   
7 0 days 00:00:03.500000    HAM  Mercedes               47.4    101  2023   
8        0 days 00:00:04    HAM  Mercedes               47.4    128  2023   
9 0 days 00:00:04.500000    HAM  Mercedes               47.4    144  2023   

  Circuito  Giri_Totali  Lunghezza_Circuito  Posizione  Giro_in_Corso  \
0   Monaco           78                3337        5.0            1.0   
1   Mo

In [22]:
  # Calcola il tempo trascorso dall'inizio del giro
lap_start_time = driver_laps['LapTime']
lap_start_time.head(5)

1127   0 days 00:01:28.585000
1128   0 days 00:01:20.527000
1129   0 days 00:01:20.105000
1130   0 days 00:01:19.356000
1131   0 days 00:01:18.736000
Name: LapTime, dtype: timedelta64[ns]

In [24]:
driver_laps.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')

In [23]:
driver_laps.head(5)

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
1127,0 days 01:03:31.805000,HAM,44,0 days 00:01:28.585000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:38.830000,...,True,Mercedes,0 days 01:02:02.950000,2023-05-28 13:03:03.927,12,5.0,False,,False,False
1128,0 days 01:04:52.332000,HAM,44,0 days 00:01:20.527000,2.0,1.0,NaT,NaT,0 days 00:00:21.226000,0 days 00:00:37.769000,...,True,Mercedes,0 days 01:03:31.805000,2023-05-28 13:04:32.782,1,5.0,False,,False,True
1129,0 days 01:06:12.437000,HAM,44,0 days 00:01:20.105000,3.0,1.0,NaT,NaT,0 days 00:00:21.081000,0 days 00:00:37.472000,...,True,Mercedes,0 days 01:04:52.332000,2023-05-28 13:05:53.309,1,5.0,False,,False,True
1130,0 days 01:07:31.793000,HAM,44,0 days 00:01:19.356000,4.0,1.0,NaT,NaT,0 days 00:00:20.805000,0 days 00:00:37.445000,...,True,Mercedes,0 days 01:06:12.437000,2023-05-28 13:07:13.414,1,5.0,False,,False,True
1131,0 days 01:08:50.529000,HAM,44,0 days 00:01:18.736000,5.0,1.0,NaT,NaT,0 days 00:00:20.748000,0 days 00:00:37.010000,...,True,Mercedes,0 days 01:07:31.793000,2023-05-28 13:08:32.770,1,5.0,False,,False,True


In [ ]:
print(df.head(50))

                Timestamp Driver  Scuderia  Track_Temperature  Speed  Anno  \
0         0 days 00:00:00    HAM  Mercedes               47.4      0  2023   
1  0 days 00:00:00.500000    HAM  Mercedes               47.4      0  2023   
2         0 days 00:00:01    HAM  Mercedes               47.4     12  2023   
3  0 days 00:00:01.500000    HAM  Mercedes               47.4     30  2023   
4         0 days 00:00:02    HAM  Mercedes               47.4     50  2023   
5  0 days 00:00:02.500000    HAM  Mercedes               47.4     67  2023   
6         0 days 00:00:03    HAM  Mercedes               47.4     78  2023   
7  0 days 00:00:03.500000    HAM  Mercedes               47.4    101  2023   
8         0 days 00:00:04    HAM  Mercedes               47.4    128  2023   
9  0 days 00:00:04.500000    HAM  Mercedes               47.4    144  2023   
10        0 days 00:00:05    HAM  Mercedes               47.4    164  2023   
11 0 days 00:00:05.500000    HAM  Mercedes               47.4   

In [12]:
df.to_csv(f'data {driver} at {circuito} on {anno}.csv', index=False)  # Salva i dati in un file CSV